In [11]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('white')
sns.set(font_scale=1.5)

data_dir = r'C:\Users\James\toronto-open-data-portal'

nbrhood_shp_path = os.path.join(data_dir, 'toronto-nbrhoods-wgs84', 'Neighbourhoods.shp')

#nbrhood_covid_count_path = os.path.join(data_dir, 'toronto-covid-nbrhood-count-060620.csv')
nbrhood_covid_count_path = os.path.join(data_dir, 'toronto-covid-nbrhood-count-060820.csv')
#nbrhood_covid_count_path = os.path.join(data_dir, 'toronto-covid-nbrhood-count-061520.csv')
#nbrhood_covid_count_path = os.path.join(data_dir, 'toronto-covid-nbrhood-count-062220.csv')


## Loading Demographic and COVID-19 Data

### **1. Neighbourhood profiles:** demographic information about the 140 Toronto Social Planning Neighbourhoods. Data is from the 2016 Census


In [12]:
# reading in the demographic info per neighbourhood

demo_profiles = pd.read_csv(os.path.join(data_dir, 'toronto-nbrhoods-wgs84', 'nbrhood-profiles-2016.csv'),
                            encoding='ISO-8859-1')


In [13]:
demo_profiles.head(3)

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"


In [14]:
profile_cols = list(zip(demo_profiles.T.loc['Category', :], demo_profiles.T.loc['Characteristic', :]))

In [15]:
demo_profiles = demo_profiles.iloc[:,6:]

In [16]:
index = pd.MultiIndex.from_tuples(profile_cols, names=['Category', 'Characteristic'])

In [17]:
demo_profiles.set_index(index, inplace=True)
demo_profiles = demo_profiles.T
demo_profiles.head(3)

Category                     Neighbourhood Information                       \
Characteristic                    Neighbourhood Number TSNS2020 Designation   
Agincourt North                                    129       No Designation   
Agincourt South-Malvern West                       128       No Designation   
Alderwood                                           20       No Designation   

Category                           Population                   \
Characteristic               Population, 2016 Population, 2011   
Agincourt North                        29,113           30,279   
Agincourt South-Malvern West           23,757           21,988   
Alderwood                              12,054           11,904   

Category                                                  \
Characteristic               Population Change 2011-2016   
Agincourt North                                   -3.90%   
Agincourt South-Malvern West                       8.00%   
Alderwood                                          1.30%   

Category                                              \
Characteristic               Total private dwellings   
Agincourt North                                9,371   
Agincourt South-Malvern West                   8,535   
Alderwood                                      4,732   

Category                                                                    \
Characteristic               Private dwellings occupied by usual residents   
Agincourt North                                                      9,120   
Agincourt South-Malvern West                                         8,136   
Alderwood                                                            4,616   

Category                                                              \
Characteristic               Population density per square kilometre   
Agincourt North                                                3,929   
Agincourt South-Malvern West                                   3,034   
Alderwood                                                      2,435   

Category                                                     \
Characteristic               Land area in square kilometres   
Agincourt North                                        7.41   
Agincourt South-Malvern West                           7.83   
Alderwood                                              4.95   

Category                                            ...  \
Characteristic               Children (0-14 years)  ...   
Agincourt North                              3,840  ...   
Agincourt South-Malvern West                 3,075  ...   
Alderwood                                    1,760  ...   

Category                                    Mobility  \
Characteristic                     External migrants   
Agincourt North                                  605   
Agincourt South-Malvern West                     490   
Alderwood                                         70   

Category                                                                            \
Characteristic               Total - Mobility status 5 years ago - 25% sample data   
Agincourt North                                                          27,490      
Agincourt South-Malvern West                                             22,325      
Alderwood                                                                11,370      

Category                                                             \
Characteristic                 Non-movers   Movers     Non-migrants   
Agincourt North                    18,865    8,610            5,445   
Agincourt South-Malvern West       13,565    8,775            5,610   
Alderwood                           8,235    3,130            2,200   

Category                                                           \
Characteristic                   Migrants       Internal migrants   
Agincourt North                     3,170                     880   
Agincourt South-Malvern West        3,145           

### **2. Covid19 Counts per Neighbourhood:** number of cases in each of the 14 Toronto Social Planning Neighbourhoods, downloaded 06-08-2020

In [68]:
covid_counts = pd.read_csv(nbrhood_covid_count_path, encoding='ISO-8859-1')
covid_counts.head()

,Neighbourhood Name,Case Count
0,Yorkdale-Glen Park,136
1,York University Heights,307
2,Yonge-St.Clair,21
3,Yonge-Eglinton,13
4,Wychwood,73


In [69]:
# The covid data contains a record of counts for missing addresses/postal codes
covid_counts.sort_values(by='Case Count', ascending=False)

,Neighbourhood Name,Case Count
140,Missing Address/Postal Code,2139
94,Glenfield-Jane Heights,354
15,West Humber-Clairville,352
106,Downsview-Roding-CFB,339
58,Mount Olive-Silverstone-Jamestown,328
...,...,...
125,Blake-Jones,12
28,St.Andrew-Windfields,12
42,Playter Estates-Danforth,9
111,Danforth,8


## Loading Toronto Map Data

In [70]:
nbrhood_geo_data = gpd.read_file(nbrhood_shp_path)
nbrhood_geo_cols = pd.read_csv(os.path.join(data_dir, 'toronto-nbrhoods-wgs84',
                                           'Neighbourhoods_fields.csv'), encoding='ISO-8859-1')

In [71]:
nbrhood_geo_data.rename(columns=dict(zip(nbrhood_geo_cols.field, nbrhood_geo_cols.name)), inplace=True)

In [72]:
nbrhood_geo_data.head(3)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,5601,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,5602,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,5603,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."


In [73]:
# remove the area code and trailing whitespace from the area names
nbrhood_geo_data['AREA_NAME'] = nbrhood_geo_data.AREA_NAME.str.replace('\s\(\d+\)', '')

In [74]:
set(nbrhood_geo_data.AREA_NAME).difference(set(covid_counts['Neighbourhood Name']))

{'Briar Hill-Belgravia',
 'Cabbagetown-South St.James Town',
 'Danforth East York',
 'North St.James Town'}

In [100]:
# identified differences in the covid nbrhood names using Pandas' dataframe.col.str.extractall(REGEX) function
covid_counts.loc[[50, 120, 110, 124]]

,Neighbourhood Name,Case Count
50,North St.James Town,145
120,Cabbagetown-South St.James Town,34
110,Danforth East York,20
124,Briar Hill-Belgravia,75


In [101]:
# renaming the neighbourhood names in the covid data to the Toronto shapefile conventions
covid_counts.loc[50, 'Neighbourhood Name'] = 'North St.James Town'
covid_counts.loc[120, 'Neighbourhood Name'] = 'Cabbagetown-South St.James Town'
covid_counts.loc[110, 'Neighbourhood Name'] = 'Danforth East York'
covid_counts.loc[124, 'Neighbourhood Name'] = 'Briar Hill-Belgravia'

In [102]:
covid_counts.rename(columns={'Neighbourhood Name':'AREA_NAME'}, inplace=True)

In [103]:
covid_counts.set_index('AREA_NAME', inplace=True)
nbrhood_geo_data.set_index('AREA_NAME', inplace=True)

In [106]:
nbrhood_geo_data.join(covid_counts).head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry,Case Count
AREA_NAME,,,,,,,,,,,,,,,,
Wychwood,5601,25886861,25926662,49885,94,94,Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680...",73
Yonge-Eglinton,5602,25886820,25926663,49885,100,100,Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704...",13
Yonge-St.Clair,5603,25886834,25926664,49885,97,97,Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680...",21
York University Heights,5604,25886593,25926665,49885,27,27,York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((-79.50529 43.75987, -79.50488 43.759...",307
Yorkdale-Glen Park,5605,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((-79.43969 43.70561, -79.44011 43.705...",136
